In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm
from utils import preprocess, train_utils
import numpy as np
import os
import pandas as pd
import pathlib
import pickle
import tensorboardX
tqdm.pandas()

In [ ]:
%%time
path_to_base = pathlib.Path('/opt/data/anuj/other/quora/processed')
df_train = pd.read_csv(str(path_to_base / 'train.csv'))
df_val = pd.read_csv(str(path_to_base / 'val.csv'))
df_test = pd.read_csv(str(path_to_base / 'test.csv'))

In [ ]:
df_train.head()

## Load word vectors

In [ ]:
%%time
with open('/opt/data/anuj/other/quora/glove.840B.300d/pickle.pkl', 'rb') as f:
    embeddings = pickle.load(f)

## Get the dataloader!

In [ ]:
BATCH_SIZE = 1
DEVICE = 1

In [ ]:
# Train
dataset_train = preprocess.QuoraDataset(df_train, word_vectors=embeddings, preprocessor=preprocess.preprocessor)
dataloader_train = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

# Val
dataset_val = preprocess.QuoraDataset(df_val, word_vectors=embeddings, preprocessor=preprocess.preprocessor)
dataloader_val = DataLoader(dataset=dataset_val, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

print(len(dataset_train), len(dataloader_train))
print(len(dataset_val), len(dataloader_val))

In [ ]:
it = iter(dataloader_train)
next(it)['final_sentence1']

### Class Weights

In [ ]:
df_train.target.value_counts()

In [ ]:
bincounts = df_train.target.value_counts().values
bincounts = 1. / bincounts
bincounts = bincounts / bincounts.sum()

In [ ]:
bincounts

## Model

In [ ]:
from utils.models_baseline import LSTM

In [ ]:
model = LSTM().cuda(device=DEVICE)

n_params = 0
for param in model.parameters():
    if param.requires_grad: n_params += np.prod(param.size())

print(f'Model had {n_params} params')

## Define loss, optim

In [ ]:
import torch

In [ ]:
# Optim and loss
weights = torch.from_numpy(bincounts.astype(np.float32)).cuda(device=DEVICE)
loss_func = torch.nn.NLLLoss(weight=weights).cuda(device=DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model_str = 'quora-insencere-lstm-1.1'

model_dir = '/opt/weights/insincere/{}'.format(model_str)
log_dir = '/home/anuj/code/tensorboard-logs/{}'.format(model_str)

os.makedirs(model_dir, exist_ok=False)  # MEANT TO FAIL IF IT ALREADY EXISTS
writer = tensorboardX.SummaryWriter(log_dir)

## Train

In [ ]:
train_utils.train(
    model=model, dataloader_train=dataloader_train, dataloader_val=dataloader_val,
    loss_func=loss_func, optimizer=optimizer, device=DEVICE, writer=writer,
)